# Create EEG Sentences


In [6]:
import pickle
import os

def load_sample_groups(filepath="../data/sample_groups.pkl"):
    """Loads the grouped samples dictionary from a pickle file.

    Args:
        filepath (str): The path to the pickle file containing the sample groups dictionary.
                        Defaults to 	"sample_groups.pkl	" in the current directory.

    Returns:
        dict: The loaded sample_groups dictionary, or None if loading fails.
              Structure: {char: {set_name: [chunk1, chunk2, ...], ...}, ...}
              where each chunk is a list of numpy arrays (samples).
    """
    print(f"Attempting to load sample groups from: {filepath}")
    if not os.path.exists(filepath):
        print(f"Error: File not found at {filepath}. Please run the grouping script first.")
        return None
    
    try:
        with open(filepath, "rb") as f:
            loaded_groups = pickle.load(f)
        print("Successfully loaded sample groups dictionary.")
        # Optional: Add a check to ensure it	"s a dictionary
        if isinstance(loaded_groups, dict):
            return loaded_groups
        else:
            print(f"Error: Loaded object is not a dictionary (type: {type(loaded_groups)}). Returning None.")
            return None
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None
    except pickle.UnpicklingError:
        print(f"Error: Could not unpickle data from {filepath}. File might be corrupted.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during loading: {e}")
        return None


sample_groups_data = load_sample_groups()

if sample_groups_data:
    print("\n--- Example Access --- ")
    target_char = 	"A" # Example character
    target_set = 	"set1" # Example set

    if target_char in sample_groups_data and target_set in sample_groups_data[target_char]:
        groups_for_set = sample_groups_data[target_char][target_set]
        num_chunks = len(groups_for_set)
        print(f"Character {target_char}, Set {target_set} has {num_chunks} chunks.")
        
        if num_chunks > 0:
            first_chunk = groups_for_set[0] # Get the first chunk (which is a list of samples)
            num_samples_in_chunk = len(first_chunk)
            print(f"  First chunk contains {num_samples_in_chunk} samples.")
            
            if num_samples_in_chunk > 0:
                first_sample_in_chunk = first_chunk[0] # Get the first sample (numpy array)
                print(f"    Shape of the first sample in the first chunk: {first_sample_in_chunk.shape}")
                # You can now work with 	"first_sample_in_chunk	" or the entire 	"first_chunk	" list
    else:
        print(f"Could not find data for Character {target_char} and Set {target_set}.")


Attempting to load sample groups from: ../data/sample_groups.pkl
Successfully loaded sample groups dictionary.

--- Example Access --- 
Character A, Set set1 has 4 chunks.
  First chunk contains 30 samples.
    Shape of the first sample in the first chunk: (78, 64)
